# Koch fractal

The Koch curve is one of the earliest fractals that has been described. The construction is illustrated in the figure below.

![construction][1]

The construction steps reads:
1. divide line into three segments of equal length
2. divide middle segment in two subsegments of equal length
3. rotate subsegments by $60\deg$ to create a triangle
4. repeat step 1 one every straight line

[1]: http://ecademy.agnesscott.edu/~lriddle/ifs/kcurve/construction.gif

In [119]:
from numba import jit
from plotly import offline as py
from plotly import graph_objs as go

py.init_notebook_mode(connected=True)

As a start we implement the Koch transformation in the `koch` function. The Koch transformation takes a straight line with start coordinate $(x_0,y_0)$ and end coordinate $(x_1,y_1)$ and returns five coordinates $(x_0,y_0),\dots,(x_4,y_4)$ that visualized show a equilateral triangle in the middle segment.

In [125]:
@jit(nopython=True)
def koch(x, y):
    assert len(x) == len(y) == 2

    u = []
    v = []

    for i in [0.0, 1/3, 1/2, 2/3, 1.0]:
        u0 = x[0] + i * (x[1] - x[0])
        v0 = y[0] + i * (y[1] - y[0])

        u.append(u0)
        v.append(v0)

    a = 2 * 3**(1/2)
        
    u[2] += (y[0] - y[1]) / a
    v[2] += (x[1] - x[0]) / a
        
    return u, v

In the `for` loop the `koch` function creates the necessary segments. In the last part the `koch` function translates the coordinates of the center segment in order to create a equilateral triangle.

The expression used for said translation can be obtained through, $$
R(\pi/2)\frac{\boldsymbol{x}_1-\boldsymbol{x}_0}{||\boldsymbol{x}_1-\boldsymbol{x}_0||}h,
$$ wherein $R(\theta)\in\mathbb{R}^{2\times2}$ denotes the rotation matrix in two dimensions and $h$ is the height of an equilateral triangle given by, $$
h
=
\frac{\sqrt{3}}{2}b
=
\frac{\sqrt{3}}{2}\frac{||\boldsymbol{x}_1-\boldsymbol{x}_0||}{3},
$$ with $b$ being the base length.

In [126]:
x0, y0 = [0, 1], [0, 0]
x1, y1 = koch(x0, y0)
x2, y2 = koch(x1[1:3], y1[1:3])

layout = go.Layout(
    title='Koch Transform',
    xaxis=dict(title='x'),
    yaxis=dict(title='y', scaleanchor='x'),
)

figure = go.Figure([
    go.Scatter(x=x0, y=y0, mode='markers+lines', name='n = 0'),
    go.Scatter(x=x1, y=y1, mode='markers+lines', name='n = 1'),
    go.Scatter(x=x2, y=y2, mode='markers+lines', name='n = 2'),
], layout)

py.iplot(figure)

Our implementation of the Koch transform in `koch` seems to work out as expected. Now in order to obtain the complete Koch curve we need to recursively apply `koch` to every straight line. Fortunately these are given by $(x_i,y_i),(x_{i+1},y_{i+1})$.

In [131]:
@jit
def koch_curve(x, y, n):
    assert len(x) == len(y)
    
    u = []
    v = []
    
    for i in range(len(x) - 1):
        a, b = koch(x[i:i+2], y[i:i+2])
        
        u += a
        v += b
    
    if n > 0:
        return koch_curve(u, v, n-1)
        
    return u, v    

By applying `koch_curve` recursively we avoid keeping track of old and new segments.

In [132]:
x0, y0 = [0, 1], [0, 0]
x1, y1 = koch_curve(x0, y0, 0)
x2, y2 = koch_curve(x0, y0, 1)
x3, y3 = koch_curve(x0, y0, 2)
x4, y4 = koch_curve(x0, y0, 3)

layout = go.Layout(
    title='Koch Curve',
    xaxis=dict(title='x'),
    yaxis=dict(title='y', scaleanchor='x'),
)

figure = go.Figure([
    go.Scatter(x=x0, y=y0, mode='markers+lines', name='n = 0'),
    go.Scatter(x=x1, y=y1, mode='markers+lines', name='n = 1'),
    go.Scatter(x=x2, y=y2, mode='markers+lines', name='n = 2'),
    go.Scatter(x=x3, y=y3, mode='markers+lines', name='n = 3'),
    go.Scatter(x=x4, y=y4, mode='markers+lines', name='n = 4'),
], layout)

py.iplot(figure)

`koch_curve` seems to work as expected!

As a next step we want to create Koch snowflakes. In principle it should be possible to use `koch_curve` with a sequence of coordinates that shape a triangle.

In [134]:
x0, y0 = [0.0, .5, 1.0, 0.0], [0.0, .5*3**.5, 0.0, 0.0]
x1, y1 = koch_curve(x0, y0, 0)
x2, y2 = koch_curve(x0, y0, 1)
x3, y3 = koch_curve(x0, y0, 2)
x4, y4 = koch_curve(x0, y0, 3)

layout = go.Layout(
    title='Koch Snowflake',
    xaxis=dict(title='x'),
    yaxis=dict(title='y', scaleanchor='x'),
)

figure = go.Figure([
    go.Scatter(x=x0, y=y0, mode='markers+lines', name='n = 0'),
    go.Scatter(x=x1, y=y1, mode='markers+lines', name='n = 1'),
    go.Scatter(x=x3, y=y3, mode='markers+lines', name='n = 3'),
    go.Scatter(x=x4, y=y4, mode='markers+lines', name='n = 4'),
], layout)

py.iplot(figure)

As a final exercise we want to estimate the correlation dimension. As a remainder the correlation dimension is defined as, $$
C(\epsilon)
=
\frac{1}{n^2}\sum^n_{i\neq j}\theta\left(\epsilon-||\boldsymbol{x}_i-\boldsymbol{x}_j||\right),
$$ wherein $\theta(\cdot)$ denotes the Heaviside step function and $n$ is the number of coordinates. Intuitively the correlation dimension counts the number of fractal points that are closer to each other than $\epsilon$.

In [139]:
x, y = koch_curve([0.0, .5, 1.0, 0.0], [0.0, .5*3**.5, 0.0, 0.0], 9)

In [186]:
x = np.array(x)
y = np.array(y)

z = np.stack([x, y]).T
d = np.linalg.norm(z, axis=1)

In [205]:
eps = np.array([1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0])
cor = np.array([np.sum(d < eps) / z.shape[0]**2 for eps in eps])

In [207]:
logeps = np.log(eps)
logcor = np.log(cor)

dim, off = np.polyfit(logeps, logcor, 1)

layout = go.Layout(
    title='Koch Dimension',
    xaxis=dict(title='log ε'),
    yaxis=dict(title='log C(ε)'),
    showlegend=True,
)

figure = go.Figure([
    go.Scatter(x=logeps, y=logcor, mode='markers+lines', name='cor'),
    go.Scatter(x=logeps, y=off + dim*logeps, mode='markers+lines', name='fit'),
], layout)

py.iplot(figure)

In [208]:
dim

1.416399736458714

In [209]:
np.log(4) / np.log(3)

1.2618595071429148

With $n=9$ iterations we find that the correlation dimension yields $d\approx1.41$. In the literature the fractal dimension is reported with $d=\ln4/\ln3\approx1.26$, thus, we are not that far off.